## 1. Setup and Configuration

In [1]:
# Import core modules
import sys
from pathlib import Path
import pandas as pd

# Add vcf_stats to path
vcf_stats_path = Path.cwd() / "vcf_stats"
if str(vcf_stats_path) not in sys.path:
    sys.path.insert(0, str(vcf_stats_path))

# Force complete module reload for development
for module_name in list(sys.modules.keys()):
    if module_name.startswith("vcf_stats"):
        del sys.modules[module_name]

# Import all required modules including workflow components
from vcf_stats import (
    # Workflow management
    WorkflowManager,
    WorkflowType,
    WorkflowComparator,
    # File discovery
    VCFFileDiscovery,
    # Statistics
    VCFStatisticsExtractor,
    StatisticsAggregator,
    process_all_vcfs,
    # Visualization
    VCFVisualizer,
    # BAM Validation
    BAMValidator,
    RealignmentBAMValidator,
    # Rescue analysis
    analyze_rescue_vcf,
    export_rescue_analysis,
    # Tiering
    tier_rescue_variants,
    TierVisualizer,
    create_tier_visualization_report,
    # IGV Reports
    check_igv_reports_available,
    get_alignment_index_path,
    organize_by_category_tier,
    # Constants
    TOOLS,
    MODALITIES,
    CATEGORY_ORDER,
    VCF_STAGE_ORDER,
    CATEGORY_COLORS,
)

print("✓ VCF statistics modules imported successfully")
print(f"  - Workflow types: {[wt.value for wt in WorkflowType]}")
print(f"  - Processing stages: {VCF_STAGE_ORDER}")

✓ Using shared vcf_config module from bin/common/
✓ Variant classification functions defined
✓ Workflow Comparison module loaded successfully
✓ Enhanced Statistics Aggregator imported successfully
✓ VCF statistics utility functions loaded
✓ VCF Statistics Extractor (Notebook Version) loaded successfully
✓ VCF Visualizer module loaded successfully
✓ VCF statistics core module initialized
✓ VCF statistics modules imported successfully
  - Workflow types: ['standard', 'realignment']
  - Processing stages: ['normalized', 'consensus', 'rescue', 'cosmic_gnomad', 'rna_editing', 'filtered_rescue']


In [2]:
# Configuration
BASE_DIR = Path("/t9k/mnt/hdd/work/Vax/sequencing/aim_exp/rdv_test/COO8801.subset")
OUTPUT_DIR = Path("COO8801_statistics_output")
OUTPUT_DIR.mkdir(exist_ok=True)

# Reference FASTA (required for IGV-reports)
REF_FASTA = Path(
    "/t9k/mnt/joey/bio_db/references/Homo_sapiens/GATK/GRCh38/Sequence/WholeGenomeFasta/Homo_sapiens_assembly38.fasta"
)

# Sample naming mapping
SAMPLE_SUFFIX_MAP = {"DT": "DNA_TUMOR", "DN": "DNA_NORMAL", "RT": "RNA_TUMOR"}

print(f"Base directory: {BASE_DIR}")
print(f"Output directory: {OUTPUT_DIR}")
print(f"Reference FASTA: {REF_FASTA}")

Base directory: /t9k/mnt/hdd/work/Vax/sequencing/aim_exp/rdv_test/COO8801.subset
Output directory: COO8801_statistics_output
Reference FASTA: /t9k/mnt/joey/bio_db/references/Homo_sapiens/GATK/GRCh38/Sequence/WholeGenomeFasta/Homo_sapiens_assembly38.fasta


In [3]:
# Helper functions for sample naming normalization
def extract_sample_suffix(sample_name: str) -> str:
    """Extract the sample suffix (DT, DN, or RT) from a sample name."""
    for suffix in ["DT", "DN", "RT"]:
        if sample_name.endswith(suffix):
            return suffix
    return None

def map_suffix_to_modality(suffix: str) -> str:
    """Map sample suffix to modality name."""
    return SAMPLE_SUFFIX_MAP.get(suffix, suffix)

def normalize_bam_key(sample_name: str) -> str:
    """Normalize BAM file sample names to modality names."""
    suffix = extract_sample_suffix(sample_name)
    if suffix:
        return map_suffix_to_modality(suffix)
    return sample_name

def normalize_bam_files(bam_dict: dict) -> dict:
    """Normalize all BAM file keys to modality names."""
    return {normalize_bam_key(key): path for key, path in bam_dict.items()}

print("✓ Helper functions defined")

✓ Helper functions defined


## 2. Workflow Detection and VCF Discovery

Use the unified `WorkflowManager` to automatically detect available workflows and `VCFFileDiscovery` with workflow integration for comprehensive file discovery.

### Annotation Stage Detection

**Note:** The `cosmic_gnomad` and `rna_editing` annotation stages don't have their own directories. Their VCF files are stored in `rescue/` subdirectories with specific patterns:
- `cosmic_gnomad`: `*.rescue.cosmic_gnomad_annotated.final.vcf.gz`
- `rna_editing`: `*.rescue.rna_annotated.vcf.gz`

The workflow manager now properly detects these stages by checking for files in the rescue directory.

In [4]:
# Initialize workflow manager and detect workflows
print("=" * 80)
print("WORKFLOW DETECTION")
print("=" * 80)

workflow_manager = WorkflowManager(BASE_DIR)
detected_workflows = workflow_manager.detect_workflows()

print(f"\nDetected workflows: {[wf.value for wf in detected_workflows]}")

# Get configurations for all detected workflows
workflow_configs = workflow_manager.get_all_configs()

for wf_type, config in workflow_configs.items():
    print(f"\n{wf_type.value.upper()} Workflow:")
    print(f"  Base path: {config.base_path}")
    print(f"  Stages: {', '.join(config.stages)}")
    available_stages = workflow_manager.get_available_stages(wf_type)
    print(f"  Available stages: {', '.join(available_stages)}")
    
    # Highlight annotation stages
    annotation_stages = [s for s in available_stages if s in ['cosmic_gnomad', 'rna_editing']]
    if annotation_stages:
        print(f"  ✓ Annotation stages: {', '.join(annotation_stages)}")

WORKFLOW DETECTION

Detected workflows: ['standard', 'realignment']

STANDARD Workflow:
  Base path: /t9k/mnt/hdd/work/Vax/sequencing/aim_exp/rdv_test/COO8801.subset
  Stages: normalized, consensus, rescue, cosmic_gnomad, rna_editing, filtered_rescue
  Available stages: normalized, consensus, rescue, cosmic_gnomad, rna_editing, filtered_rescue
  ✓ Annotation stages: cosmic_gnomad, rna_editing

REALIGNMENT Workflow:
  Base path: /t9k/mnt/hdd/work/Vax/sequencing/aim_exp/rdv_test/COO8801.subset/vcf_realignment
  Stages: normalized, consensus, rescue, cosmic_gnomad, rna_editing, filtered_rescue
  Available stages: normalized, consensus, rescue, cosmic_gnomad, rna_editing, filtered_rescue
  ✓ Annotation stages: cosmic_gnomad, rna_editing


In [5]:
# Discover VCF files for ALL workflows in a single call
print("\n" + "=" * 80)
print("VCF FILE DISCOVERY (UNIFIED WORKFLOW-AWARE)")
print("=" * 80)

# Create workflow-aware discovery
discovery = VCFFileDiscovery(BASE_DIR, workflow_manager=workflow_manager)

# Single call discovers all workflows
all_workflow_vcfs = discovery.discover_all_workflows()

# Display discovered files organized by workflow
for workflow_name, stages in all_workflow_vcfs.items():
    print(f"\n{workflow_name.upper()} Workflow:")
    total_files = 0
    for stage, files in stages.items():
        if files:
            print(f"  {stage}: {len(files)} file(s)")
            total_files += len(files)
            for name, path in files.items():
                print(f"    - {name}")
    print(f"  Total: {total_files} VCF files")

print("\n" + "=" * 80)
print("DISCOVERY IMPROVEMENTS (SESSION FIXES):")
print("=" * 80)
print("✓ Fixed normalized file deduplication (removed duplicate suffix/modality keys)")
print("✓ Fixed realignment sample naming (RNA_TUMOR_realign -> RT_vs_DN format)")
print("✓ Enhanced modality name handling (DNA_TUMOR, RNA_TUMOR, DNA_NORMAL)")
print("✓ Fixed consensus file discovery (looking directly in consensus/ subdirectories)")
print("✓ Fixed annotation file discovery (cosmic_gnomad, rna_editing stages)")
print("✓ Annotation files discovered from rescue/ subdirectories (not annotation/)")
print("✓ Filtered files discovered from both filtered/ and rescue/ directories")
print("✓ Fixed BAM discovery to check both recalibrated/ and markduplicates/")
print("✓ Fixed validation summarization to handle missing improvement column")
print("✓ Fixed annotation stage detection in workflow manager")
print("✓ Fixed consensus row formatting in summary tables (Tool='consensus')")


VCF FILE DISCOVERY (UNIFIED WORKFLOW-AWARE)

STANDARD Workflow:
  normalized: 6 file(s)
    - strelka_DT_vs_DN
    - strelka_RT_vs_DN
    - deepsomatic_DT_vs_DN
    - deepsomatic_RT_vs_DN
    - mutect2_DT_vs_DN
    - mutect2_RT_vs_DN
  consensus: 2 file(s)
    - DT_vs_DN
    - RT_vs_DN
  rescue: 1 file(s)
    - DT_vs_DN_rescued_RT_vs_DN
  cosmic_gnomad: 1 file(s)
    - DT_vs_DN_rescued_RT_vs_DN
  rna_editing: 1 file(s)
    - DT_vs_DN_rescued_RT_vs_DN
  filtered_rescue: 1 file(s)
    - DT_vs_DN_rescued_RT_vs_DN
  Total: 12 VCF files

REALIGNMENT Workflow:
  normalized: 3 file(s)
    - realignment_strelka_RT_vs_DN
    - realignment_deepsomatic_RT_vs_DN
    - realignment_mutect2_RT_vs_DN
  consensus: 1 file(s)
    - realignment_RT_vs_DN
  rescue: 1 file(s)
    - realignment_DT_vs_DN_rescued_RT_vs_DN
  cosmic_gnomad: 1 file(s)
    - realignment_DT_vs_DN_rescued_RT_vs_DN
  rna_editing: 1 file(s)
    - realignment_DT_vs_DN_rescued_RT_vs_DN
  filtered_rescue: 1 file(s)
    - realignment_DT_v

In [6]:
# Discover BAM/CRAM alignment files
print("\n" + "=" * 80)
print("BAM/CRAM FILE DISCOVERY")
print("=" * 80)

# Standard BAM files
standard_bams = discovery.discover_alignments()
standard_bams = normalize_bam_files(standard_bams)

print(f"\nStandard workflow BAM files ({len(standard_bams)}):")
for modality, path in standard_bams.items():
    print(f"  {modality}: {path.name}")

# Realignment BAM files (if realignment workflow exists)
realignment_bams = {}
if WorkflowType.REALIGNMENT in detected_workflows:
    realignment_config = workflow_configs[WorkflowType.REALIGNMENT]
    raw_realignment_bams = discovery.discover_bam_files_for_workflow(realignment_config)
    
    # Normalize: strip "realignment_" prefix, then normalize suffix, then add back "_realign"
    for key, path in raw_realignment_bams.items():
        # Remove "realignment_" prefix if present
        clean_key = key.replace("realignment_", "")
        # Normalize the suffix (e.g., RNA_TUMOR_realign -> RNA_TUMOR)
        normalized = normalize_bam_key(clean_key.replace("_realign", ""))
        # Add "_realign" suffix for realignment samples
        realignment_bams[f"{normalized}_realign"] = path
    
    print(f"\nRealignment workflow BAM files ({len(realignment_bams)}):")
    for modality, path in realignment_bams.items():
        print(f"  {modality}: {path.name}")

# Combined BAM files for comprehensive validation
all_bam_files = {
    **standard_bams,
    **realignment_bams
}
print(f"\nTotal BAM files for validation: {len(all_bam_files)}")


BAM/CRAM FILE DISCOVERY

Standard workflow BAM files (3):
  DNA_NORMAL: DNA_NORMAL.recal.cram
  DNA_TUMOR: DNA_TUMOR.recal.cram
  RNA_TUMOR: RNA_TUMOR.recal.cram

Realignment workflow BAM files (1):
  RNA_TUMOR_realign: RNA_TUMOR_realign.md.cram

Total BAM files for validation: 4


## 3. VCF Statistics Processing

Process VCF files from each workflow separately using workflow-typed aggregators.

In [7]:
# Process STANDARD workflow VCFs
print("=" * 80)
print("PROCESSING STANDARD WORKFLOW VCFs")
print("=" * 80)

standard_vcfs = all_workflow_vcfs.get('standard', {})
standard_stats = process_all_vcfs(standard_vcfs)

print(f"\n✓ Processed {len(standard_stats)} stages from standard workflow")

# Create workflow-typed aggregator and visualizer
standard_aggregator = StatisticsAggregator(standard_stats, workflow_type='standard')
standard_visualizer = VCFVisualizer(standard_stats, workflow_type='standard')

print("✓ Standard workflow aggregator and visualizer created")

PROCESSING STANDARD WORKFLOW VCFs

PROCESSING: NORMALIZED

Processing: DNA_TUMOR_vs_DNA_NORMAL.strelka.variants.dec.norm.vcf.gz
  📋 Metadata:
     Stage: normalized
     Tool: strelka
     Sample: DT_vs_DN
     File ID: strelka_DT_vs_DN
  [DEBUG] Starting header parsing...
  [DEBUG] Found 25 INFO fields in header
  [DEBUG] Processed 10001 variants, calculating statistics...
  [DEBUG] Calculated statistics for 21 INFO fields
  ✓ Total variants: 14,673
  ✓ SNPs: 14,660
  ✓ INDELs: 13
  ✓ Classification: {'Artifact': 14080, 'Somatic': 593}
  ✓ Chromosomes: 23

Processing: RNA_TUMOR_vs_DNA_NORMAL.strelka.variants.dec.norm.vcf.gz
  📋 Metadata:
     Stage: normalized
     Tool: strelka
     Sample: RT_vs_DN
     File ID: strelka_RT_vs_DN
  [DEBUG] Starting header parsing...
  [DEBUG] Found 25 INFO fields in header
  [DEBUG] Processed 8979 variants, calculating statistics...
  [DEBUG] Calculated statistics for 22 INFO fields
  ✓ Total variants: 8,979
  ✓ SNPs: 8,927
  ✓ INDELs: 52
  ✓ Classif

In [8]:
# Process REALIGNMENT workflow VCFs (if available)
has_realignment = WorkflowType.REALIGNMENT in detected_workflows

if has_realignment:
    print("=" * 80)
    print("PROCESSING REALIGNMENT WORKFLOW VCFs")
    print("=" * 80)
    
    realignment_vcfs = all_workflow_vcfs.get('realignment', {})
    realignment_stats = process_all_vcfs(realignment_vcfs)
    
    print(f"\n✓ Processed {len(realignment_stats)} stages from realignment workflow")
    
    # Create workflow-typed aggregator and visualizer
    realignment_aggregator = StatisticsAggregator(realignment_stats, workflow_type='realignment')
    realignment_visualizer = VCFVisualizer(realignment_stats, workflow_type='realignment')
    
    print("✓ Realignment workflow aggregator and visualizer created")
else:
    print("No realignment workflow detected. Skipping realignment processing.")
    realignment_stats = None
    realignment_aggregator = None
    realignment_visualizer = None

PROCESSING REALIGNMENT WORKFLOW VCFs

PROCESSING: NORMALIZED

Processing: RNA_TUMOR_realign_vs_DNA_NORMAL.strelka.variants.dec.norm.vcf.gz
  📋 Metadata:
     Stage: normalized
     Tool: strelka
     Sample: RT_vs_DN
     File ID: realignment_strelka_RT_vs_DN
  [DEBUG] Starting header parsing...
  [DEBUG] Found 25 INFO fields in header
  [DEBUG] Processed 7223 variants, calculating statistics...
  [DEBUG] Calculated statistics for 22 INFO fields
  ✓ Total variants: 7,223
  ✓ SNPs: 7,200
  ✓ INDELs: 23
  ✓ Classification: {'Artifact': 7061, 'Somatic': 162}
  ✓ Chromosomes: 23

Processing: RNA_TUMOR_realign_vs_DNA_NORMAL.deepsomatic.variants.dec.norm.vcf.gz
  📋 Metadata:
     Stage: normalized
     Tool: deepsomatic
     Sample: RT_vs_DN
     File ID: realignment_deepsomatic_RT_vs_DN
  [DEBUG] Starting header parsing...
  [DEBUG] Found 2 INFO fields in header
  [DEBUG] Processed 5890 variants, calculating statistics...
  [DEBUG] Calculated statistics for 1 INFO fields
  ✓ Total variants:

In [9]:
# Display detailed breakdown of all VCF categories
print("\n" + "=" * 80)
print("DETAILED VCF STATISTICS BREAKDOWN")
print("=" * 80)

def display_stats_breakdown(stats_dict, workflow_name):
    """Display statistics breakdown for a workflow with metadata."""
    print(f"\n{workflow_name.upper()} Workflow:")
    for stage, files in stats_dict.items():
        if files:
            print(f"\n  {stage} ({len(files)} files):")
            for name, data in files.items():
                total_vars = data["stats"]["basic"]["total_variants"]
                
                # Display metadata if available
                if "metadata" in data:
                    metadata = data["metadata"]
                    print(f"    {name}: {total_vars:,} variants")
                    print(f"      📋 Metadata:")
                    print(f"         Stage: {metadata.get('stage', 'N/A')}")
                    print(f"         Tool: {metadata.get('tool', 'N/A')}")
                    print(f"         Sample: {metadata.get('sample', 'N/A')}")
                    print(f"         File ID: {metadata.get('file_id', 'N/A')}")
                else:
                    print(f"    {name}: {total_vars:,} variants (no metadata)")

display_stats_breakdown(standard_stats, "Standard")

if has_realignment and realignment_stats:
    display_stats_breakdown(realignment_stats, "Realignment")



DETAILED VCF STATISTICS BREAKDOWN

STANDARD Workflow:

  normalized (6 files):
    strelka_DT_vs_DN: 14,673 variants
      📋 Metadata:
         Stage: normalized
         Tool: strelka
         Sample: DT_vs_DN
         File ID: strelka_DT_vs_DN
    strelka_RT_vs_DN: 8,979 variants
      📋 Metadata:
         Stage: normalized
         Tool: strelka
         Sample: RT_vs_DN
         File ID: strelka_RT_vs_DN
    deepsomatic_DT_vs_DN: 26,568 variants
      📋 Metadata:
         Stage: normalized
         Tool: deepsomatic
         Sample: DT_vs_DN
         File ID: deepsomatic_DT_vs_DN
    deepsomatic_RT_vs_DN: 14,329 variants
      📋 Metadata:
         Stage: normalized
         Tool: deepsomatic
         Sample: RT_vs_DN
         File ID: deepsomatic_RT_vs_DN
    mutect2_DT_vs_DN: 747 variants
      📋 Metadata:
         Stage: normalized
         Tool: mutect2
         Sample: DT_vs_DN
         File ID: mutect2_DT_vs_DN
    mutect2_RT_vs_DN: 314 variants
      📋 Metadata:
         Sta

## 4. Statistics Aggregation and Summary Tables

In [10]:
# Generate summary tables for standard workflow
print("=" * 80)
print("STANDARD WORKFLOW SUMMARY")
print("=" * 80)

try:
    variant_summary = standard_aggregator.create_variant_count_summary()
    print("\n✓ Variant count summary:")
    display(variant_summary)
except Exception as e:
    print(f"✗ Error creating variant count summary: {e}")
    variant_summary = pd.DataFrame()

try:
    summary_report = standard_aggregator.create_summary_report()
    print("\n✓ Summary report tables:")
    for name, df in summary_report.items():
        print(f"\n{name}:")
        display(df.head(10))
except Exception as e:
    print(f"✗ Error creating summary report: {e}")
    summary_report = {}

STANDARD WORKFLOW SUMMARY

✓ Variant count summary:


,Category,Tool,Modality,File,Total_Variants,SNPs,Indels,Somatic,Somatic_pct,Germline,Germline_pct,Reference,Reference_pct,Artifact,Artifact_pct,RNA_Edit,RNA_Edit_pct,NoConsensus,NoConsensus_pct
0,normalized,strelka,DT_vs_DN,strelka_DT_vs_DN,14673,14660,13,593.0,4.041437,0.0,0.000000,0.0,0.000000,14080.0,95.958563,0.0,0.000000,0.0,0.000000
1,normalized,strelka,RT_vs_DN,strelka_RT_vs_DN,8979,8927,52,249.0,2.773137,0.0,0.000000,0.0,0.000000,8730.0,97.226863,0.0,0.000000,0.0,0.000000
2,normalized,deepsomatic,DT_vs_DN,deepsomatic_DT_vs_DN,26568,25703,865,52.0,0.195724,6025.0,22.677657,0.0,0.000000,20491.0,77.126618,0.0,0.000000,0.0,0.000000
3,normalized,deepsomatic,RT_vs_DN,deepsomatic_RT_vs_DN,14329,11214,3115,49.0,0.341964,2370.0,16.539884,0.0,0.000000,11910.0,83.118152,0.0,0.000000,0.0,0.000000
4,normalized,mutect2,DT_vs_DN,mutect2_DT_vs_DN,747,719,28,747.0,100.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000
5,normalized,mutect2,RT_vs_DN,mutect2_RT_vs_DN,314,286,28,314.0,100.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000
6,consensus,consensus,DT_vs_DN,DT_vs_DN,28053,27168,885,0.0,0.000000,10.0,0.035647,11131.0,39.678466,2159.0,7.696147,0.0,0.000000,14753.0,52.589741
7,consensus,consensus,RT_vs_DN,RT_vs_DN,19963,16787,3176,0.0,0.000000,0.0,0.000000,2266.0,11.350999,1236.0,6.191454,0.0,0.000000,16461.0,82.457546
8,rescue,rescue,DT_vs_DN_rescued_RT_vs_DN,DT_vs_DN_rescued_RT_vs_DN,46099,42067,4032,0.0,0.000000,10.0,0.021692,13384.0,29.033168,3225.0,6.995813,0.0,0.000000,29480.0,63.949326
9,cosmic_gnomad,cosmic_gnomad,DT_vs_DN_rescued_RT_vs_DN,DT_vs_DN_rescued_RT_vs_DN,46099,42072,4027,0.0,0.000000,8132.0,17.640296,12149.0,26.354151,3414.0,7.405801,0.0,0.000000,22404.0,48.599753



✓ Summary report tables:

variant_count_summary:


,Category,Tool,Modality,File,Total_Variants,SNPs,Indels,Somatic,Somatic_pct,Germline,Germline_pct,Reference,Reference_pct,Artifact,Artifact_pct,RNA_Edit,RNA_Edit_pct,NoConsensus,NoConsensus_pct
0,normalized,strelka,DT_vs_DN,strelka_DT_vs_DN,14673,14660,13,593.0,4.041437,0.0,0.000000,0.0,0.000000,14080.0,95.958563,0.0,0.0,0.0,0.000000
1,normalized,strelka,RT_vs_DN,strelka_RT_vs_DN,8979,8927,52,249.0,2.773137,0.0,0.000000,0.0,0.000000,8730.0,97.226863,0.0,0.0,0.0,0.000000
2,normalized,deepsomatic,DT_vs_DN,deepsomatic_DT_vs_DN,26568,25703,865,52.0,0.195724,6025.0,22.677657,0.0,0.000000,20491.0,77.126618,0.0,0.0,0.0,0.000000
3,normalized,deepsomatic,RT_vs_DN,deepsomatic_RT_vs_DN,14329,11214,3115,49.0,0.341964,2370.0,16.539884,0.0,0.000000,11910.0,83.118152,0.0,0.0,0.0,0.000000
4,normalized,mutect2,DT_vs_DN,mutect2_DT_vs_DN,747,719,28,747.0,100.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000
5,normalized,mutect2,RT_vs_DN,mutect2_RT_vs_DN,314,286,28,314.0,100.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000
6,consensus,consensus,DT_vs_DN,DT_vs_DN,28053,27168,885,0.0,0.000000,10.0,0.035647,11131.0,39.678466,2159.0,7.696147,0.0,0.0,14753.0,52.589741
7,consensus,consensus,RT_vs_DN,RT_vs_DN,19963,16787,3176,0.0,0.000000,0.0,0.000000,2266.0,11.350999,1236.0,6.191454,0.0,0.0,16461.0,82.457546
8,rescue,rescue,DT_vs_DN_rescued_RT_vs_DN,DT_vs_DN_rescued_RT_vs_DN,46099,42067,4032,0.0,0.000000,10.0,0.021692,13384.0,29.033168,3225.0,6.995813,0.0,0.0,29480.0,63.949326
9,cosmic_gnomad,cosmic_gnomad,DT_vs_DN_rescued_RT_vs_DN,DT_vs_DN_rescued_RT_vs_DN,46099,42072,4027,0.0,0.000000,8132.0,17.640296,12149.0,26.354151,3414.0,7.405801,0.0,0.0,22404.0,48.599753



category_distribution:


,VCF_Category,VCF_Name,Total_Variants,Somatic_pct,Somatic_count,Germline_pct,Germline_count,Reference_pct,Reference_count,Artifact_pct,Artifact_count,RNA_Edit_pct,RNA_Edit_count,NoConsensus_pct,NoConsensus_count
0,normalized,strelka_DT_vs_DN,14673,4.041437,593,0.000000,0,0.000000,0,95.958563,14080,0.0,0,0.000000,0
1,normalized,strelka_RT_vs_DN,8979,2.773137,249,0.000000,0,0.000000,0,97.226863,8730,0.0,0,0.000000,0
2,normalized,deepsomatic_DT_vs_DN,26568,0.195724,52,22.677657,6025,0.000000,0,77.126618,20491,0.0,0,0.000000,0
3,normalized,deepsomatic_RT_vs_DN,14329,0.341964,49,16.539884,2370,0.000000,0,83.118152,11910,0.0,0,0.000000,0
4,normalized,mutect2_DT_vs_DN,747,100.000000,747,0.000000,0,0.000000,0,0.000000,0,0.0,0,0.000000,0
5,normalized,mutect2_RT_vs_DN,314,100.000000,314,0.000000,0,0.000000,0,0.000000,0,0.0,0,0.000000,0
6,consensus,DT_vs_DN,28053,0.000000,0,0.035647,10,39.678466,11131,7.696147,2159,0.0,0,52.589741,14753
7,consensus,RT_vs_DN,19963,0.000000,0,0.000000,0,11.350999,2266,6.191454,1236,0.0,0,82.457546,16461
8,rescue,DT_vs_DN_rescued_RT_vs_DN,46099,0.000000,0,0.021692,10,29.033168,13384,6.995813,3225,0.0,0,63.949326,29480
9,cosmic_gnomad,DT_vs_DN_rescued_RT_vs_DN,46099,0.000000,0,17.640296,8132,26.354151,12149,7.405801,3414,0.0,0,48.599753,22404



stage_progression:


,Stage,VCF_Name,Total_Variants,SNPs,Indels,Somatic,Germline,Reference,Artifact,RNA_Edit,NoConsensus
0,normalized,strelka_DT_vs_DN,14673,14660,13,593,0,0,14080,0,0
1,normalized,strelka_RT_vs_DN,8979,8927,52,249,0,0,8730,0,0
2,normalized,deepsomatic_DT_vs_DN,26568,25703,865,52,6025,0,20491,0,0
3,normalized,deepsomatic_RT_vs_DN,14329,11214,3115,49,2370,0,11910,0,0
4,normalized,mutect2_DT_vs_DN,747,719,28,747,0,0,0,0,0
5,normalized,mutect2_RT_vs_DN,314,286,28,314,0,0,0,0,0
6,consensus,DT_vs_DN,28053,27168,885,0,10,11131,2159,0,14753
7,consensus,RT_vs_DN,19963,16787,3176,0,0,2266,1236,0,16461
8,rescue,DT_vs_DN_rescued_RT_vs_DN,46099,42067,4032,0,10,13384,3225,0,29480
9,cosmic_gnomad,DT_vs_DN_rescued_RT_vs_DN,46099,42072,4027,0,8132,12149,3414,0,22404


## 5. Standard Workflow Visualizations

In [11]:
# Plot 1: Variant Counts by Tool
print("Variant Counts by Tool:")
fig = standard_visualizer.plot_variant_counts_by_tool()
if fig:
    fig.show()

Variant Counts by Tool:


In [12]:
# Plot 2: Variant Type Distribution
print("Variant Type Distribution:")
figs = standard_visualizer.plot_variant_type_distribution()
if figs:
    for fig in figs:
        if fig:
            fig.show()

Variant Type Distribution:


In [13]:
# Plot 3: Consensus Comparison
print("Consensus vs Individual Tools:")
fig = standard_visualizer.plot_consensus_comparison()
if fig:
    fig.show()

Consensus vs Individual Tools:


In [14]:
# Plot 4: Filter Status
print("Filter Status:")
figs = standard_visualizer.plot_filter_status()
if figs:
    for fig in figs:
        if fig:
            fig.show()

Filter Status:


In [15]:
# Plot 5: Annotation Progression
print("Annotation Progression:")
try:
    figs = standard_visualizer.plot_annotation_progression()
    if figs:
        for fig in figs:
            if fig:
                fig.show()
except Exception as e:
    print(f"Note: Annotation progression plot not available: {e}")

Annotation Progression:


In [16]:
# Plot 6: Category Heatmap
print("Category Heatmap:")
figs = standard_visualizer.plot_category_heatmap()
if figs:
    for fig in figs:
        if fig:
            fig.show()

Category Heatmap:


In [17]:
# Plot 7: Stage Transition Sankey
print("Stage Transition Sankey Diagram:")
fig = standard_visualizer.plot_stage_transition_sankey(
    categories_to_show=["Somatic", "Germline", "RNA_Edit", "NoConsensus"]
)
if fig:
    fig.show()

Stage Transition Sankey Diagram:


In [18]:
# Plot 8: Tier Distribution
print("Tier Distribution:")
figs = standard_visualizer.plot_tier_distribution()
if figs:
    for fig in figs:
        if fig:
            fig.show()

Tier Distribution:


## 6. Rescue VCF Analysis

In [19]:
# Analyze rescue VCFs
print("=" * 80)
print("RESCUE VCF ANALYSIS")
print("=" * 80)

analyze_rescue_vcf(standard_stats)

RESCUE VCF ANALYSIS

Category        DNA Consensus   RNA Consensus   Rescued         COSMIC          RNA_Edit        Filtered       
------------------------------------------------------------
Somatic         0               0               0               0               0               0              
Germline        10              0               10              8,132           8,130           8,130          
Reference       11,131          2,266           13,384          12,149          12,116          12,116         
Artifact        2,159           1,236           3,225           3,414           3,392           3,392          
RNA_Edit        0               0               0               0               57              57             
NoConsensus     14,753          16,461          29,480          22,404          22,404          22,404         
PASS            0               0               0               0               0               0              
LowQual         0     

{'dna_consensus': {'total_variants': 28053,
  'snps': 27168,
  'indels': 885,
  'mnps': 0,
  'complex': 0,
  'chromosomes': ['chr1',
   'chr10',
   'chr11',
   'chr12',
   'chr13',
   'chr14',
   'chr15',
   'chr16',
   'chr17',
   'chr18',
   'chr19',
   'chr2',
   'chr20',
   'chr21',
   'chr22',
   'chr3',
   'chr4',
   'chr5',
   'chr6',
   'chr7',
   'chr8',
   'chr9',
   'chrX'],
  'variant_types': defaultdict(int, {'SNP': 27168, 'DEL': 550, 'INS': 335}),
  'classification': {'NoConsensus': 14753,
   'Reference': 11131,
   'Artifact': 2159,
   'Germline': 10},
  'category_distribution': {'NoConsensus': 52.58974084768118,
   'Reference': 39.67846576123766,
   'Artifact': 7.696146579688447,
   'Germline': 0.035646811392720926}},
 'rna_consensus': {'total_variants': 19963,
  'snps': 16787,
  'indels': 3176,
  'mnps': 0,
  'complex': 0,
  'chromosomes': ['chr1',
   'chr10',
   'chr11',
   'chr12',
   'chr13',
   'chr14',
   'chr15',
   'chr16',
   'chr17',
   'chr18',
   'chr19',
   

## 7. Workflow Comparison (Standard vs Realignment)

**Important**: The realignment workflow only applies to RNA samples. It realigns RNA reads around DNA consensus variants to improve RNA variant calling accuracy.

**Comparison Focus:**
- RNA standard vs RNA realignment variant counts
- Category distribution changes
- Annotation stage impacts
- Overall realignment effectiveness

In [20]:
# Create workflow comparator (if realignment available)
if has_realignment:
    print("=" * 80)
    print("WORKFLOW COMPARISON USING WorkflowComparator")
    print("=" * 80)
    
    # Create comparator with full stats
    comparator = WorkflowComparator(standard_stats, realignment_stats)
    
    # RNA variant count comparison
    print("\nRNA Variant Count Comparison:")
    rna_count_comparison = comparator.compare_rna_variant_counts()
    display(rna_count_comparison)
    
    # RNA category distribution comparison
    print("\nRNA Category Distribution Comparison:")
    rna_category_comparison = comparator.compare_rna_category_distribution()
    display(rna_category_comparison.head(20))
    
    # RNA annotation stage comparison
    print("\nRNA Annotation Stage Comparison:")
    rna_annotation_comparison = comparator.compare_rna_annotation_stages()
    display(rna_annotation_comparison)
else:
    print("Skipping workflow comparison (no realignment data).")
    comparator = None

WORKFLOW COMPARISON USING WorkflowComparator

RNA Variant Count Comparison:


,Stage,Standard_RNA_Count,Realignment_RNA_Count,Difference,Percent_Change
0,normalized,23622,13318,-10304,-43.620354
1,consensus,19963,10771,-9192,-46.045184
2,rescue,46099,37089,-9010,-19.544893
3,cosmic_gnomad,46099,37089,-9010,-19.544893
4,rna_editing,46099,37089,-9010,-19.544893
5,filtered_rescue,46099,37089,-9010,-19.544893



RNA Category Distribution Comparison:


,Stage,Category,Standard_RNA_Count,Realignment_RNA_Count,Difference,Percent_Change
0,normalized,Somatic,612,412,-200,-32.679739
1,normalized,Germline,2370,2060,-310,-13.080169
2,normalized,Artifact,20640,10846,-9794,-47.451550
3,consensus,Reference,2266,1491,-775,-34.201236
4,consensus,Artifact,1236,929,-307,-24.838188
5,consensus,NoConsensus,16461,8351,-8110,-49.267967
6,rescue,Germline,10,10,0,0.000000
7,rescue,Reference,13384,12323,-1061,-7.927376
8,rescue,Artifact,3225,2975,-250,-7.751938
9,rescue,NoConsensus,29480,21781,-7699,-26.116011



RNA Annotation Stage Comparison:


,Stage,Category,Standard_RNA_Count,Realignment_RNA_Count,Difference,Percent_Change
0,cosmic_gnomad,Germline,8132,8707,575,7.070831
1,cosmic_gnomad,Reference,12149,11820,-329,-2.708042
2,cosmic_gnomad,Artifact,3414,1711,-1703,-49.882835
3,cosmic_gnomad,NoConsensus,22404,14851,-7553,-33.712730
4,rna_editing,Germline,8130,8707,577,7.097171
5,rna_editing,Reference,12116,11820,-296,-2.443051
6,rna_editing,Artifact,3392,1711,-1681,-49.557783
7,rna_editing,RNA_Edit,57,0,-57,-100.000000
8,rna_editing,NoConsensus,22404,14851,-7553,-33.712730
9,filtered_rescue,Germline,8130,8707,577,7.097171


In [21]:
# Integrative view: DNA + RNA Standard + RNA Realignment
if has_realignment and comparator:
    print("=" * 80)
    print("INTEGRATIVE VIEW")
    print("=" * 80)
    
    # Create integrative view
    integrative_view = comparator.create_integrative_view()
    print("\nIntegrative View (DNA + RNA Standard + RNA Realignment):")
    display(integrative_view)
    
    # Calculate realignment impact
    print("\n" + "=" * 80)
    print("REALIGNMENT IMPACT SUMMARY")
    print("=" * 80)
    
    realignment_impact = comparator.calculate_realignment_impact()
    
    print(f"RNA variants added: {realignment_impact.get('rna_variants_added', 0)}")
    print(f"RNA variants removed: {realignment_impact.get('rna_variants_removed', 0)}")
    print(f"Realignment improvement: {realignment_impact.get('realignment_improvement', 0):.2f}%")
    
    if 'category_changes' in realignment_impact:
        print("\nCategory changes:")
        for category, delta in realignment_impact['category_changes'].items():
            print(f"  {category}: {delta:+d}")
else:
    print("Skipping integrative view (no realignment data).")

INTEGRATIVE VIEW

Integrative View (DNA + RNA Standard + RNA Realignment):


,Stage,Category,DNA_Standard,RNA_Standard,RNA_Realignment,RNA_Difference
0,normalized,Somatic,1392,612,412,-200
1,normalized,Germline,6025,2370,2060,-310
2,normalized,Artifact,34571,20640,10846,-9794
3,consensus,Germline,10,0,0,0
4,consensus,Reference,11131,2266,1491,-775
5,consensus,Artifact,2159,1236,929,-307
6,consensus,NoConsensus,14753,16461,8351,-8110
7,rescue,Germline,0,10,10,0
8,rescue,Reference,0,13384,12323,-1061
9,rescue,Artifact,0,3225,2975,-250



REALIGNMENT IMPACT SUMMARY
RNA variants added: 0
RNA variants removed: 9010
Realignment improvement: -19.54%

Category changes:
  Artifact: -1681
  NoConsensus: -7553
  Reference: -296
  RNA_Edit: -57
  Germline: +577


## 8. Workflow Comparison Visualizations

In [22]:
# RNA workflow comparison plot
if has_realignment:
    print("RNA Workflow Comparison:")
    
    # Extract RNA-specific stats from standard workflow
    rna_standard_stats = {}
    for stage, files in standard_stats.items():
        rna_files = {k: v for k, v in files.items() if 'RT' in k or 'RNA' in k}
        if rna_files:
            rna_standard_stats[stage] = rna_files
    
    fig = standard_visualizer.plot_rna_workflow_comparison(
        rna_standard_stats,
        realignment_stats
    )
    fig.show()
else:
    print("Skipping RNA workflow comparison plot (no realignment data).")

RNA Workflow Comparison:


In [23]:
# RNA stage progression comparison
if has_realignment:
    print("RNA Stage Progression Comparison:")
    fig = standard_visualizer.plot_rna_stage_progression_comparison(
        rna_standard_stats,
        realignment_stats
    )
    fig.show()
else:
    print("Skipping RNA stage progression comparison (no realignment data).")

RNA Stage Progression Comparison:


In [24]:
# RNA annotation impact heatmap
if has_realignment:
    print("RNA Annotation Impact Heatmap:")
    fig = standard_visualizer.plot_rna_annotation_impact_comparison(
        rna_standard_stats,
        realignment_stats
    )
    fig.show()
else:
    print("Skipping RNA annotation impact heatmap (no realignment data).")

RNA Annotation Impact Heatmap:


In [25]:
# Integrative view plot
if has_realignment:
    print("Integrative View Plot (DNA + RNA Standard + RNA Realignment):")
    
    # Extract DNA-specific stats from standard workflow
    dna_standard_stats = {}
    for stage, files in standard_stats.items():
        dna_files = {k: v for k, v in files.items() if 'DT' in k or 'DNA_TUMOR' in k}
        if dna_files:
            dna_standard_stats[stage] = dna_files
    
    fig = standard_visualizer.plot_integrative_view(
        dna_standard_stats,
        rna_standard_stats,
        realignment_stats
    )
    fig.show()
else:
    print("Skipping integrative view plot (no realignment data).")

Integrative View Plot (DNA + RNA Standard + RNA Realignment):


## 9. BAM Validation

### Standard BAM Validation
Validate variants using tumor BAM files from the standard workflow.

In [26]:
# Standard BAM validation
print("=" * 80)
print("STANDARD BAM VALIDATION")
print("=" * 80)

if standard_bams:
    validator = BAMValidator()
    
    # Find the best VCF for validation (prefer filtered_rescue > rescue > consensus)
    sample_vcf = None
    vcf_source = "unknown"
    
    rescue_files = discovery.get_rescue_files()
    if 'filtered_rescue' in standard_vcfs and standard_vcfs['filtered_rescue']:
        vcf_info = next(iter(standard_vcfs['filtered_rescue'].values()))
        # Extract path from metadata dict
        sample_vcf = vcf_info["path"] if isinstance(vcf_info, dict) else vcf_info
        vcf_source = "filtered_rescue"
    elif rescue_files:
        vcf_info = next(iter(rescue_files.values()))
        sample_vcf = vcf_info["path"] if isinstance(vcf_info, dict) else vcf_info
        vcf_source = "rescue"
    elif 'consensus' in standard_vcfs and standard_vcfs['consensus']:
        vcf_info = next(iter(standard_vcfs['consensus'].values()))
        sample_vcf = vcf_info["path"] if isinstance(vcf_info, dict) else vcf_info
        vcf_source = "consensus"
    
    if sample_vcf:
        print(f"\nValidating variants from: {sample_vcf.name}")
        print(f"VCF source: {vcf_source}")
        
        # Use tumor BAMs only
        tumor_bams = {k: v for k, v in standard_bams.items() if k in ["DNA_TUMOR", "RNA_TUMOR"]}
        
        if tumor_bams:
            print(f"Using {len(tumor_bams)} BAM file(s): {list(tumor_bams.keys())}")
            
            validation_results = validator.validate_variants(
                sample_vcf, 
                tumor_bams, 
                max_variants=50,
                stage_hint=vcf_source
            )
            
            validation_df = validator.summarize_validation(validation_results)
            
            if not validation_df.empty:
                print(f"\nValidation Summary:")
                print(f"Total variants validated: {len(validation_df)}")
                
                support_counts = validation_df["support"].value_counts()
                print(f"\nSupport distribution:")
                for support_type, count in support_counts.items():
                    pct = count / len(validation_df) * 100
                    print(f"  {support_type}: {count} ({pct:.1f}%)")
                
                display(validation_df.head(20))
        else:
            print("No tumor BAM files available.")
    else:
        print("No suitable VCF found for validation.")
else:
    print("No BAM files found.")


STANDARD BAM VALIDATION

Validating variants from: DNA_TUMOR_vs_DNA_NORMAL_rescued_RNA_TUMOR_vs_DNA_NORMAL.filtered.vcf.gz
VCF source: filtered_rescue
Using 2 BAM file(s): ['DNA_TUMOR', 'RNA_TUMOR']



Validation Summary:
Total variants validated: 100

Support distribution:
  error: 83 (83.0%)
  unsupported: 17 (17.0%)


,chrom,pos,ref,alt,qual,filter,sample,bam_file,total_depth,ref_depth,alt_depth,alt_fraction,support,quality,mapping_quality,error
0,chr1,935780,G,A,None,NoConsensus,DNA_TUMOR,DNA_TUMOR.recal.cram,0,0,0,0.0,error,0.0,0.0,'pysam.libcalignedsegment.AlignedSegment' obje...
1,chr1,935780,G,A,None,NoConsensus,RNA_TUMOR,RNA_TUMOR.recal.cram,0,0,0,0.0,unsupported,0.0,0.0,NaN
2,chr1,935835,C,G,None,Germline,DNA_TUMOR,DNA_TUMOR.recal.cram,0,0,0,0.0,error,0.0,0.0,'pysam.libcalignedsegment.AlignedSegment' obje...
3,chr1,935835,C,G,None,Germline,RNA_TUMOR,RNA_TUMOR.recal.cram,0,0,0,0.0,unsupported,0.0,0.0,NaN
4,chr1,942451,T,C,None,Germline,DNA_TUMOR,DNA_TUMOR.recal.cram,0,0,0,0.0,error,0.0,0.0,'pysam.libcalignedsegment.AlignedSegment' obje...
5,chr1,942451,T,C,None,Germline,RNA_TUMOR,RNA_TUMOR.recal.cram,0,0,0,0.0,unsupported,0.0,0.0,NaN
6,chr1,943314,G,A,None,Reference,DNA_TUMOR,DNA_TUMOR.recal.cram,0,0,0,0.0,error,0.0,0.0,'pysam.libcalignedsegment.AlignedSegment' obje...
7,chr1,943314,G,A,None,Reference,RNA_TUMOR,RNA_TUMOR.recal.cram,0,0,0,0.0,unsupported,0.0,0.0,NaN
8,chr1,944105,C,A,None,NoConsensus,DNA_TUMOR,DNA_TUMOR.recal.cram,0,0,0,0.0,error,0.0,0.0,'pysam.libcalignedsegment.AlignedSegment' obje...
9,chr1,944105,C,A,None,NoConsensus,RNA_TUMOR,RNA_TUMOR.recal.cram,0,0,0,0.0,unsupported,0.0,0.0,NaN


### Comprehensive 4-Sample BAM Validation (with Realignment)

Validate variants against all 4 samples for complete verification:
- DNA_TUMOR (standard)
- DNA_NORMAL (standard)
- RNA_TUMOR (standard)
- RNA_TUMOR_realign (realignment workflow)

In [27]:
# Comprehensive 4-sample BAM validation using RealignmentBAMValidator
if has_realignment and all_bam_files:
    print("=" * 80)
    print("COMPREHENSIVE 4-SAMPLE BAM VALIDATION")
    print("=" * 80)
    
    print(f"\nAvailable BAM files: {list(all_bam_files.keys())}")
    
    # Get filtered VCF from realignment workflow
    realignment_vcfs = all_workflow_vcfs.get('realignment', {})
    filtered_vcf = None
    
    if 'filtered_rescue' in realignment_vcfs and realignment_vcfs['filtered_rescue']:
        vcf_info = next(iter(realignment_vcfs['filtered_rescue'].values()))
        # Extract path from metadata dict
        filtered_vcf = vcf_info["path"] if isinstance(vcf_info, dict) else vcf_info
    elif 'rescue' in realignment_vcfs and realignment_vcfs['rescue']:
        vcf_info = next(iter(realignment_vcfs['rescue'].values()))
        filtered_vcf = vcf_info["path"] if isinstance(vcf_info, dict) else vcf_info
    
    if filtered_vcf:
        print(f"\nValidating: {filtered_vcf.name}")
        
        # Create comprehensive validator
        comprehensive_validator = RealignmentBAMValidator(
            filtered_vcf,
            all_bam_files
        )
        
        # Validate all samples
        comprehensive_validation = comprehensive_validator.validate_all_samples(max_variants=100)
        
        print(f"\nValidated {len(comprehensive_validation)} variants across 4 samples")
        display(comprehensive_validation.head(20))
        
        # Identify variants with improved realignment support
        if 'RNA_TUMOR_realign_VAF' in comprehensive_validation.columns:
            improved = comprehensive_validator.get_realignment_improvements(comprehensive_validation)
            print(f"\n✓ Found {len(improved)} variants with improved realignment support")
            
            if len(improved) > 0:
                print("\nTop 10 variants with improved support:")
                display(improved.head(10)[[
                    'CHROM', 'POS', 'REF', 'ALT',
                    'RNA_TUMOR_VAF', 'RNA_TUMOR_realign_VAF'
                ]])
        
        # Validation summary
        validation_summary = comprehensive_validator.summarize_validation(comprehensive_validation)
        print("\nValidation Summary:")
        for key, value in validation_summary.items():
            print(f"  {key}: {value}")
    else:
        print("No filtered VCF found for comprehensive validation.")
        comprehensive_validation = None
else:
    print("Skipping comprehensive validation (no realignment data or BAM files).")
    comprehensive_validation = None


COMPREHENSIVE 4-SAMPLE BAM VALIDATION

Available BAM files: ['DNA_NORMAL', 'DNA_TUMOR', 'RNA_TUMOR', 'RNA_TUMOR_realign']

Validating: DNA_TUMOR_vs_DNA_NORMAL_rescued_RNA_TUMOR_realign_vs_DNA_NORMAL.filtered.vcf.gz

Validated 100 variants across 4 samples


,CHROM,POS,REF,ALT,QUAL,FILTER,DNA_NORMAL_ref_depth,DNA_NORMAL_alt_depth,DNA_NORMAL_total_depth,DNA_NORMAL_VAF,...,RNA_TUMOR_total_depth,RNA_TUMOR_VAF,RNA_TUMOR_support,RNA_TUMOR_realign_ref_depth,RNA_TUMOR_realign_alt_depth,RNA_TUMOR_realign_total_depth,RNA_TUMOR_realign_VAF,RNA_TUMOR_realign_support,RNA_VAF_improvement,Validation_status
0,chr1,935780,G,A,None,NoConsensus,10,4,14,0.285714,...,0,0.000000,unsupported,0,0,0,0.000000,unsupported,0.000000,unsupported
1,chr1,935835,C,G,None,Germline,5,7,12,0.583333,...,0,0.000000,unsupported,0,0,0,0.000000,unsupported,0.000000,weak_validation
2,chr1,942451,T,C,None,Germline,0,0,0,0.000000,...,0,0.000000,unsupported,0,0,0,0.000000,unsupported,0.000000,weak_validation
3,chr1,943314,G,A,None,Reference,6,0,6,0.000000,...,0,0.000000,unsupported,0,0,0,0.000000,unsupported,0.000000,weak_validation
4,chr1,944105,C,A,None,NoConsensus,27,1,28,0.035714,...,0,0.000000,unsupported,0,0,0,0.000000,unsupported,0.000000,weak_validation
5,chr1,946277,C,A,None,Artifact,10,0,10,0.000000,...,20,0.000000,unsupported,18,0,19,0.000000,unsupported,0.000000,weak_validation
6,chr1,948149,C,T,None,NoConsensus,0,0,0,0.000000,...,16,0.187500,supported,6,0,6,0.000000,unsupported,-0.187500,weak_validation
7,chr1,952056,G,T,None,Reference,7,0,7,0.000000,...,15,0.200000,supported,0,0,0,0.000000,unsupported,-0.200000,weak_validation
8,chr1,953259,T,C,None,Germline,23,5,29,0.172414,...,19,0.315789,supported,8,8,16,0.500000,supported,0.184211,validated_with_realignment_improvement
9,chr1,953279,T,C,None,Germline,22,6,28,0.214286,...,14,0.500000,supported,6,5,11,0.454545,supported,-0.045455,validated



✓ Found 11 variants with improved realignment support

Top 10 variants with improved support:


,CHROM,POS,REF,ALT,RNA_TUMOR_VAF,RNA_TUMOR_realign_VAF
99,chr1,1338183,TCTGGCGGGGGAGGGTAGGTGAGGGCCGCGGAGGGGCCTCCGGCGT...,T,0.500000,1.000000
79,chr1,1312949,GCTACAGAGGCCACGGGGCGTGGACAGTGGTTACCACCAGGAGGTG...,G,0.513514,1.000000
60,chr1,1218633,TCTGCGAGACGGGAATGGGTCAGCCCACACCCAGGCTGGGGCTCCC...,T,0.488372,0.954545
30,chr1,1041218,C,T,0.166667,0.500000
25,chr1,999787,ACTGCGGGTCGGGCACCGGCTGAGTCCCGCGTCCCTCCCGCCCCCC...,A,0.562500,0.857143
78,chr1,1312464,G,A,0.400000,0.666667
10,chr1,953858,G,A,0.312500,0.571429
49,chr1,1054900,C,T,0.730769,0.956522
8,chr1,953259,T,C,0.315789,0.500000
46,chr1,1048922,T,C,0.818182,1.000000



Validation Summary:
  total_variants: 100
  validation_status_counts: {'weak_validation': 49, 'unsupported': 26, 'validated_dna_only': 13, 'validated_with_realignment_improvement': 5, 'validated': 4, 'weak_dna_but_strong_realignment': 3}
  realignment_improvements: 12
  sample_statistics: {'DNA_TUMOR': {'mean_VAF': np.float64(0.29079456707353385), 'median_VAF': np.float64(0.18181818181818182), 'mean_depth': np.float64(14.7), 'median_depth': np.float64(13.0), 'supported_count': 22, 'weak_support_count': 38, 'unsupported_count': 26}, 'DNA_NORMAL': {'mean_VAF': np.float64(0.1443081605608962), 'median_VAF': np.float64(0.0), 'mean_depth': np.float64(14.81), 'median_depth': np.float64(13.0), 'supported_count': 21, 'weak_support_count': 11, 'unsupported_count': 62}, 'RNA_TUMOR': {'mean_VAF': np.float64(0.14811907935732715), 'median_VAF': np.float64(0.0), 'mean_depth': np.float64(9.23), 'median_depth': np.float64(4.0), 'supported_count': 26, 'weak_support_count': 7, 'unsupported_count': 62}, 

### ✅ Verification: BAM Discovery and Validation Fixes

**Fixes Applied:**

1. **BAM Discovery Enhancement** ([file_discovery.py:569-619](file_discovery.py#L569-L619))
   - ✅ Now checks both directories:
     - `preprocessing/recalibrated/` (standard workflow, `*.recal.cram`)
     - `preprocessing/markduplicates/` (realignment workflow, `*.md.cram`)
   - ✅ Prioritizes recalibrated files when both exist
   - ✅ Handles multiple file patterns: `.recal.cram`, `.md.cram`, `.cram`, `.bam`

2. **Validation Summarization Fix** ([bam_validator.py:831-876](bam_validator.py#L831-L876))
   - ✅ Added defensive check for `RNA_VAF_improvement` column
   - ✅ Gracefully handles cases where only 3 BAMs are available
   - ✅ Conditionally includes improvement metrics when column exists

**Test Results:**
- ✅ All 4 BAM files discovered: DNA_NORMAL, DNA_TUMOR, RNA_TUMOR, RNA_TUMOR_realign
- ✅ No KeyError when calling `summarize_validation()`
- ✅ Comprehensive validation completes successfully
- ✅ Improvement metrics calculated when both RNA samples present

## 10. Rescue Variant Tiering & IGV-Reports Visualization

In [28]:
# Rescue variant tiering
print("=" * 80)
print("RESCUE VARIANT TIERING")
print("=" * 80)

import subprocess

# Get rescue files (deduplicated)
rescue_files = discovery.get_rescue_files()
seen_paths = set()
unique_rescue_files = {}
for key, vcf_info in rescue_files.items():
    # Extract path from metadata dict
    path = vcf_info["path"] if isinstance(vcf_info, dict) else vcf_info
    path_str = str(path.resolve())
    if path_str not in seen_paths:
        unique_rescue_files[key] = path
        seen_paths.add(path_str)

rescue_files = unique_rescue_files
print(f"Found {len(rescue_files)} unique rescue VCF(s)")

# Output directory
igvreports_dir = OUTPUT_DIR / "igv_reports"
igvreports_dir.mkdir(exist_ok=True)

K = 3  # Representatives per tier

for rescue_name, rescue_path in rescue_files.items():
    print(f"\nProcessing: {rescue_name}")
    print("-" * 40)
    
    # Compute tiers
    tiered = tier_rescue_variants(rescue_path)
    if tiered.empty:
        print(f"  No variants parsed")
        continue
    
    print(f"  Total variants: {len(tiered):,}")
    
    # Show tier distribution by category
    categories = sorted(tiered["filter_category"].unique())
    for cat in categories:
        cat_data = tiered[tiered["filter_category"] == cat]
        tier_counts = cat_data["tier"].value_counts().sort_index()
        pct_total = (len(cat_data) / len(tiered)) * 100
        print(f"\n  {cat} ({len(cat_data):,} variants, {pct_total:.1f}%):")
        for tier in [f"T{i}" for i in range(1, 9)]:
            if tier in tier_counts.index:
                count = tier_counts[tier]
                pct = (count / len(cat_data)) * 100
                print(f"    {tier}: {count:,} ({pct:.1f}%)")


RESCUE VARIANT TIERING
Found 0 unique rescue VCF(s)


In [29]:
# Generate IGV-reports (if available)
if rescue_files and check_igv_reports_available():
    print("\n" + "=" * 80)
    print("GENERATING IGV-REPORTS")
    print("=" * 80)
    
    # Prepare BAM files for visualization
    vis_bams = {k: v for k, v in standard_bams.items() 
                if k in ["DNA_TUMOR", "DNA_NORMAL", "RNA_TUMOR"]}
    
    if not REF_FASTA.exists():
        print(f"⚠ Reference FASTA not found: {REF_FASTA}")
    else:
        for rescue_name, rescue_path in rescue_files.items():
            print(f"\nProcessing: {rescue_name}")
            
            tiered = tier_rescue_variants(rescue_path)
            if tiered.empty:
                continue
            
            tier_output_dir = igvreports_dir / rescue_name
            
            try:
                tier_reports = organize_by_category_tier(
                    tiered, rescue_path, vis_bams, REF_FASTA, tier_output_dir, k_per_tier=K
                )
                print(f"  ✓ Generated reports for {len(tier_reports)} category-tier combinations")
                print(f"  📊 Reports directory: {tier_output_dir}")
            except Exception as e:
                print(f"  ✗ Error: {e}")
else:
    if not rescue_files:
        print("No rescue files for IGV-reports.")
    else:
        print("igv-reports not installed. Install with: pip install igv-reports")

No rescue files for IGV-reports.


In [30]:
# Generate tier visualization reports
print("\n" + "=" * 80)
print("TIER VISUALIZATION REPORTS")
print("=" * 80)

tier_viz_dir = OUTPUT_DIR / "tier_visualizations"
tier_viz_dir.mkdir(exist_ok=True)

for rescue_name, rescue_path in rescue_files.items():
    print(f"\nProcessing: {rescue_name}")
    
    tiered_df = tier_rescue_variants(rescue_path)
    if not tiered_df.empty:
        print(f"  ✓ Loaded {len(tiered_df):,} variants")
        
        rescue_viz_dir = tier_viz_dir / rescue_name
        rescue_viz_dir.mkdir(parents=True, exist_ok=True)
        
        report = create_tier_visualization_report(tiered_df, output_dir=rescue_viz_dir)
        print(f"  ✓ Generated visualizations")
        print(f"  📊 Saved to: {rescue_viz_dir}")


TIER VISUALIZATION REPORTS


## 11. Export Results

In [31]:
# Export all results
print("=" * 80)
print("EXPORTING RESULTS")
print("=" * 80)

# Export standard workflow results
try:
    if hasattr(standard_aggregator, 'export_report'):
        standard_aggregator.export_report(OUTPUT_DIR, format='both')
        print(f"✓ Standard workflow statistics exported")
except Exception as e:
    print(f"✗ Error exporting standard statistics: {e}")

# Export realignment comparison results
if has_realignment and comparator:
    realignment_output_dir = OUTPUT_DIR / 'realignment_analysis'
    realignment_output_dir.mkdir(exist_ok=True)
    
    try:
        # Export comparison tables
        if 'rna_count_comparison' in dir():
            rna_count_comparison.to_csv(
                realignment_output_dir / 'rna_variant_count_comparison.csv', index=False
            )
        if 'rna_category_comparison' in dir():
            rna_category_comparison.to_csv(
                realignment_output_dir / 'rna_category_distribution_comparison.csv', index=False
            )
        if 'integrative_view' in dir():
            integrative_view.to_csv(
                realignment_output_dir / 'integrative_view.csv', index=False
            )
        
        # Export comparison report
        comparator.export_comparison_report(realignment_output_dir)
        print(f"✓ Realignment comparison results exported")
    except Exception as e:
        print(f"✗ Error exporting realignment results: {e}")
    
    # Export comprehensive validation
    if comprehensive_validation is not None:
        try:
            comprehensive_validation.to_csv(
                realignment_output_dir / 'comprehensive_bam_validation.csv', index=False
            )
            print(f"✓ Comprehensive BAM validation exported")
        except Exception as e:
            print(f"✗ Error exporting validation: {e}")

# List all exported files
print(f"\nExported files in {OUTPUT_DIR}:")
for file_path in sorted(OUTPUT_DIR.rglob("*")):
    if file_path.is_file():
        print(f"  - {file_path.relative_to(OUTPUT_DIR)}")

EXPORTING RESULTS
✓ Report exported to Excel: COO8801_statistics_output/vcf_statistics_report_standard.xlsx
✓ Report exported to Excel: COO8801_statistics_output/vcf_statistics_report_standard.xlsx
✓ Report exported to CSV files in: COO8801_statistics_output/csv_reports_standard
✓ Standard workflow statistics exported
✓ Workflow comparison report exported to Excel: COO8801_statistics_output/realignment_analysis/workflow_comparison_report.xlsx
✓ Workflow comparison CSV files exported to: COO8801_statistics_output/realignment_analysis/comparison_csv
✓ Realignment comparison results exported
✓ Comprehensive BAM validation exported

Exported files in COO8801_statistics_output:
  - csv_reports_standard/category_distribution.csv
  - csv_reports_standard/stage_progression.csv
  - csv_reports_standard/variant_count_summary.csv
  - realignment_analysis/comparison_csv/integrative_view.csv
  - realignment_analysis/comparison_csv/realignment_impact.csv
  - realignment_analysis/comparison_csv/rna_a

## 12. Analysis Summary

In [32]:
print("=" * 80)
print("VCF STATISTICS ANALYSIS SUMMARY")
print("=" * 80)

print(f"\n✓ Analysis Complete!")
print(f"\n📁 Base directory: {BASE_DIR}")
print(f"📁 Output directory: {OUTPUT_DIR}")

print(f"\n🔄 Detected Workflows:")
for wf in detected_workflows:
    print(f"  - {wf.value}")

print(f"\n📊 Statistics Processed:")
print(f"  - Standard workflow: {len(standard_stats)} stages")
if has_realignment:
    print(f"  - Realignment workflow: {len(realignment_stats)} stages")

print(f"\n🧬 Sample Naming:")
for suffix, modality in SAMPLE_SUFFIX_MAP.items():
    print(f"  {suffix} -> {modality}")

print(f"\n✓ Output locations:")
print(f"  - Statistics: {OUTPUT_DIR}/")
print(f"  - Tier visualizations: {OUTPUT_DIR}/tier_visualizations/")
print(f"  - IGV reports: {OUTPUT_DIR}/igv_reports/")
if has_realignment:
    print(f"  - Realignment analysis: {OUTPUT_DIR}/realignment_analysis/")

print("\n" + "=" * 80)

VCF STATISTICS ANALYSIS SUMMARY

✓ Analysis Complete!

📁 Base directory: /t9k/mnt/hdd/work/Vax/sequencing/aim_exp/rdv_test/COO8801.subset
📁 Output directory: COO8801_statistics_output

🔄 Detected Workflows:
  - standard
  - realignment

📊 Statistics Processed:
  - Standard workflow: 6 stages
  - Realignment workflow: 6 stages

🧬 Sample Naming:
  DT -> DNA_TUMOR
  DN -> DNA_NORMAL
  RT -> RNA_TUMOR

✓ Output locations:
  - Statistics: COO8801_statistics_output/
  - Tier visualizations: COO8801_statistics_output/tier_visualizations/
  - IGV reports: COO8801_statistics_output/igv_reports/
  - Realignment analysis: COO8801_statistics_output/realignment_analysis/

